In [1]:
from pyspark.sql import SparkSession

In [31]:
spark = SparkSession.builder \
    .appName("M4 Attack Analysis") \
    .getOrCreate()

# kafka_messages.txt contains the log messages from the Kafka topic
file_path = 'kafka_messages.txt' 
rdd = spark.sparkContext.textFile(file_path)

In [27]:
# our log file that contains the suspected attack information
log_path = 'logfile_security.txt' 
suspected_rdd = spark.sparkContext.textFile(log_path) \
    .filter(lambda line: 'Movie' in line)
    
def get_movie_id(line):
    tokens = line.split(' ')
    index = tokens.index('has')
    return ''.join(tokens[1:index])

suspected_rdd = suspected_rdd.map(lambda line: get_movie_id(line))
suspected_movie_set = set(suspected_rdd.collect())

In [44]:

transformed_rdd = rdd.filter(lambda line: '/rate/' in line.split(',')[2]) \
    .map(lambda line: line.split(',')[2].split('/')[2].split('=')) \
    .map(lambda tokens: (tokens[0], tokens[1])) \
    .filter(lambda x: x[0] in suspected_movie_set) \
    .groupByKey() \
    .mapValues(list)

In [45]:
print(len(transformed_rdd.collect()))

79


In [64]:
def get_rating_dict(ratings):
    dict = {}
    for rating in ratings:
        rating = rating[:-1]
        if not (rating).isnumeric():
            continue
        if rating in dict:
            dict[rating] += 1
        else:
            dict[rating] = 1
    return dict

THRESHOLD = 10
evaluation_rdd = transformed_rdd.filter(lambda x: len(x[1]) > THRESHOLD) \
    .map(lambda x: (x[0], get_rating_dict(x[1])))

print(len(evaluation_rdd.collect()))
for i in evaluation_rdd.collect():
    print(i)

14
('spirited+away+2001', {'4': 923, '5': 498, '3': 179, '2': 7})
('alexander+2004', {'1': 29, '3': 1})
('the+shining+1980', {'4': 520, '5': 226, '3': 125, '2': 7})
('the+shawshank+redemption+1994', {'5': 6573, '4': 7113, '3': 906, '2': 23, '1': 1})
('inception+2010', {'3': 319, '4': 1352, '5': 702, '2': 11, '1': 1})
('confessions+2010', {'5': 7, '4': 23, '3': 3})
('iron+man+2008', {'4': 635, '5': 195, '3': 209, '2': 9})
('the+sting+1973', {'3': 9, '5': 16, '4': 28})
('an+american+crime+2007', {'4': 19, '5': 3, '3': 7, '2': 1})
('grey+gardens+1975', {'4': 21, '3': 13, '5': 8})
('star+wars+1977', {'4': 1054, '5': 643, '3': 201, '2': 8})
('sabotage+1936', {'5': 6, '4': 15, '2': 1, '3': 3})
('le+cercle+rouge+1970', {'4': 38, '3': 9, '5': 7})
('mind+game+2004', {'4': 11, '5': 7, '3': 2})
